# Experiment: Zugriff auf ein PDF File via Langhain und Mystral

Quellen
- https://medium.com/@vikastiwari708409/how-to-use-gpt4all-llms-with-langchain-to-work-with-pdf-files-f0f0becadcb6


In [ ]:
# Install langchain
!pip install langchain

# Install vectorStore
!pip install faiss-cpu

# Install gpt4all
!pip install gpt4all

# Install huggingfaceHub
!pip install huggingface-hub

# Install PyPdf for working with PDFs
!pip install pypdf

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain import PromptTemplate, LLMChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import GPT4All
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.callbacks.base import BaseCallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from pathlib import Path


documents = PyPDFLoader('./Resources/homematic-ip-anwenderhandbuch.pdf').load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024,chunk_overlap=64)
texts = text_splitter.split_documents(documents)
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
faiss_index = FAISS.from_documents(texts, embeddings)
faiss_index.save_local("./Output/langchain-csv_vector_faiss.db")


# Set your query here manually
question = "Nenne mir einige Details zu Home Connect Plus"
matched_docs = faiss_index.similarity_search(question, 4)
context = ""
for doc in matched_docs:
    context = context + doc.page_content + " \n\n "

template = """
Please use the following context to answer questions.
Context: {context}
 - -
Question: {question}
Answer: Let's think step by step."""

callback_manager = BaseCallbackManager([StreamingStdOutCallbackHandler()])
llm = GPT4All(model=f"{Path.home()}/Models/mistral-7b-openorca.Q4_0.gguf", 
              callback_manager=callback_manager, 
              verbose=True, 
              n_threads=8
              )
prompt = PromptTemplate(template=template, input_variables=["context", "question"]).partial(context=context)
llm_chain = LLMChain(prompt=prompt, llm=llm)
print(llm_chain.run(question))

/home/kosta/miniconda3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Exception ignored on calling ctypes callback function: <function LLModel._prompt_callback at 0x7f107820dee0>
Traceback (most recent call last):
  File "/home/kosta/miniconda3/envs/langchain/lib/python3.11/site-packages/gpt4all/_pyllmodel.py", line 434, in _prompt_callback
    @staticmethod

KeyboardInterrupt: 
